In [ ]:
def calculate_vpt_8(df, price_column='close'):
    df['Price_Change_8'] = df[price_column].pct_change()
    df['VPT_8'] = (df['Price_Change_8'] * df['volume']).cumsum()
    return df

def add_signal_line_8(df, window=14):
    df['Signal_Line_8'] = df['VPT_8'].rolling(window=window).mean()
    return df

def add_moving_averages_8(df, short_window=5, long_window=20, price_column='close'):
    df['Short_MA_8'] = df[price_column].rolling(window=short_window).mean()
    df['Long_MA_8'] = df[price_column].rolling(window=long_window).mean()
    return df

def add_rsi_8(df, window=14, price_column='close'):
    delta = df[price_column].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df['RSI_8'] = 100 - (100 / (1 + rs))
    return df

def generate_trading_signals_8(df, vpt_threshold=0.005, volume_threshold=1e4, rsi_threshold=50, rsi_diff_threshold=1):
    df['signals'] = np.where(
        (df['VPT_8'] > df['Signal_Line_8']) &
        (df['VPT_8'].diff() > vpt_threshold) &
        (df['volume'] > volume_threshold) &
        (df['RSI_8'] > rsi_threshold) &
        (df['RSI_8'].diff() > rsi_diff_threshold) &
        (df['Short_MA_8'] > df['Long_MA_8']),
        1,
        np.where(
            (df['VPT_8'] < df['Signal_Line_8']) &
            (df['VPT_8'].diff() < -vpt_threshold) &
            (df['volume'] > volume_threshold) &
            (df['RSI_8'] < rsi_threshold) &
            (df['RSI_8'].diff() < -rsi_diff_threshold) &
            (df['Short_MA_8'] < df['Long_MA_8']),
            -1,
            0
        )
    )
    return df

def strategy_8(df, vpt_window=14, signal_window=14, short_window=5, long_window=20, rsi_window=14):
    df = calculate_vpt_8(df, price_column='close')
    df = add_signal_line_8(df, window=signal_window)
    df = add_moving_averages_8(df, short_window=short_window, long_window=long_window, price_column='close')
    df = add_rsi_8(df, window=rsi_window, price_column='close')
    df = generate_trading_signals_8(df)
    
    
    return df
